# 19) Learning more about Pandas

Related references:

- [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/index.html)
- [Python for Data Analysis, 2nd Edition](https://www.safaribooksonline.com/library/view/python-for-data/9781491957653/) 

### Connecting the last few lectures

The last lectures on databases have some commonalities with Pandas DataFrames. When would you prefer to use a database?


## Arithmetic and Data Alignment

Arithmetic with Databases has some similar behavior to automatic outer join on the index labels. We did some work with this last Pandas lecture; let's have a little review before moving on.

In [1]:
import pandas as pd
import numpy as np

df1 = pd.DataFrame(np.arange(12.).reshape((3, 4)),
                   columns=list('abcd'))
df2 = pd.DataFrame(np.arange(20.).reshape((4, 5)),
                   columns=list('abcde'))
df2.loc[1, 'b'] = np.nan
df1

,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0


In [2]:
df2

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,4.0
1,5.0,NaN,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [3]:
df1 + df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,NaN,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


If you want to add arguments, you'll need to use built-in methods, not just the character representation. For example:

In [4]:
df1.add(df2, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,5.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


### Pandas DataFrame arithmatic

The "r\*" version of each method does the same action in reverse order. For example, 
df1.rdiv(other, axis='index') is equivalent to other.div(df1, axis='index')

| Method | Description |
|-----|-----|
| add, radd | Methods for addition (+) |
| sub, rsub | Methods for subtraction (-) |
| div, rdiv | Methods for division (/) |
| floordiv, rfloordiv | Methods for floor division (//) |
| mul, rmul | Methods for multiplication (\*) |
| pow, rpow | Methods for exponentiation (\*\*) |

### Operations between series and dataframes: remember broadcasting!

Let's look at an example:

In [5]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),
                     columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [6]:
series = frame.iloc[0]
series

b    0.0
d    1.0
e    2.0
Name: Utah, dtype: float64

What do you expect from the following?

In [7]:
frame - series

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


In [11]:
series2 = pd.Series(range(3), index=['b', 'e', 'f'])
frame + series2

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


By default, broadcasting is done over indices, matching on columns. If you want the opposite, specify so with an arithmetic method.

In [9]:
series3 = frame['d']
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [16]:
series3

Utah       1.0
Ohio       4.0
Texas      7.0
Oregon    10.0
Name: d, dtype: float64

In [17]:
frame.sub(series3, axis='index')

,b,d,e
Utah,-1.0,0.0,1.0
Ohio,-1.0,0.0,1.0
Texas,-1.0,0.0,1.0
Oregon,-1.0,0.0,1.0


There are many other functions that you might want to perform on dataframes, and quick Internet searches will give you the correct syntax. For example:

In [18]:
frame.min()

b    0.0
d    1.0
e    2.0
dtype: float64

In [21]:
frame.min(axis=1)

Utah      0.0
Ohio      3.0
Texas     6.0
Oregon    9.0
dtype: float64

In [22]:
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [23]:
frame.sort_index()

,b,d,e
Ohio,3.0,4.0,5.0
Oregon,9.0,10.0,11.0
Texas,6.0,7.0,8.0
Utah,0.0,1.0,2.0


In [24]:
frame.sort_index(axis=1, ascending=False)

,e,d,b
Utah,2.0,1.0,0.0
Ohio,5.0,4.0,3.0
Texas,8.0,7.0,6.0
Oregon,11.0,10.0,9.0


In [25]:
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])
obj.sort_values()

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

### Function Application and Mapping

NumPy ufuncs (element-wise array methods) also work with pandas objects:

In [26]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,0.311655,-1.518622,-0.365164
Ohio,-1.129441,-0.185656,-0.478807
Texas,0.428517,0.451911,0.607829
Oregon,-0.617946,0.658378,1.212348


In [29]:
def f(x):
    return x.max() - x.min()

frame.apply(f)

b    1.557959
d    2.177001
e    1.691155
dtype: float64

Here the function f, which computes the difference between the maximum and minimum of a Series, is invoked once on each column in 'frame'. The result is a Series having the columns of 'frame' as its index.

If you pass `axis='columns'` to apply, the function will be invoked once per row instead:

In [30]:
frame.apply(f, axis='columns')

Utah      1.830277
Ohio      0.943785
Texas     0.179311
Oregon    1.830294
dtype: float64

The function passed to apply need not return a scalar value; it can also return a Series with multiple values:

In [33]:
def f(x):
    return pd.Series([x.min(), x.max()], index=['min', 'max'])
frame.apply(f)

,b,d,e
min,-1.129441,-1.518622,-0.478807
max,0.428517,0.658378,1.212348


**Note**: Many of the most common array statistics (like sum and mean) are DataFrame methods, so using apply is not necessary.

### A personal favorite DataFrame method:

In [37]:
frame.describe()

,b,d,e
count,4.000000,4.000000,4.000000
mean,-0.251804,-0.148497,0.244051
std,0.749362,0.981523,0.809032
min,-1.129441,-1.518622,-0.478807
25%,-0.745820,-0.518898,-0.393575
50%,-0.153146,0.133127,0.121332
75%,0.340871,0.503528,0.758959
max,0.428517,0.658378,1.212348


In [38]:
obj = pd.Series(['a', 'a', 'b', 'c'] * 4)

In [39]:
obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object

### A list of helpful built-in descriptive and summary statistic methods

| Method | Description |
|-----|-----|
| count | Number of non-NA values |
| describe | Compute set of summary statistics for Series or each DataFrame column |
| min, max | Compute minimum and maximum values |
| argmin, argmax | Compute index locations (integers) at which minimum or maximum value obtained, respectively |
| idxmin, idxmax | Compute index labels at which minimum or maximum value obtained, respectively |
| quantile | Compute sample quantile ranging from 0 to 1 |
| sum | Sum of values |
| mean | Mean of values |
| median | Arithmetic median (50% quantile) of values |
| mad | Mean absolute deviation from mean value |
| prod | Product of all values |
| var | Sample variance of values |
| std | Sample standard deviation of values |
| skew | Sample skewness (third moment) of values |
| kurt | Sample kurtosis (fourth moment) of values |
| cumsum | Cumulative sum of values |
| cummin, cummax | Cumulative minimum or maximum of values, respectively |
| cumprod | Cumulative product of values |
| diff | Compute first arithmetic difference (useful for time series) |
| pct_change | Compute percent changes |

These functions have built-in methods for handling missing data, so a different version of them is not needed when there are null (e.g. NaN) entries.

In [40]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5],
                   [np.nan, np.nan], [0.75, -1.3]],
                  index=['a', 'b', 'c', 'd'],
                  columns=['one', 'two'])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


Calling DataFrame’s sum method returns a Series containing column sums:

In [41]:
df.sum()

one    9.25
two   -5.80
dtype: float64

In [42]:
df.sum(axis='columns')

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

Note that NA values are excluded unless the entire slice (row or column in this case) is NA. This can be disabled with the skipna option:

In [43]:
df.mean(axis='columns', skipna=False)

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

### Unique Values, Value Counts, and Membership

Another class of related methods extracts information about the values contained in a one-dimensional Series. To illustrate these, consider this example:

In [44]:
obj = pd.Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])

uniques = obj.unique()
uniques 

array(['c', 'a', 'd', 'b'], dtype=object)

The unique values are not necessarily returned in sorted order, but could be sorted after the fact if needed (`uniques.sort()`). Relatedly, value_counts computes a Series containing value frequencies:

In [45]:
obj.value_counts()

a    3
c    3
b    2
d    1
dtype: int64

The Series is sorted by value in descending order as a convenience. value_counts is also available as a top-level pandas method that can be used with any array or sequence:

In [46]:
my_list = ['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c']
pd.value_counts(my_list, sort=False)

c    3
a    3
b    2
d    1
dtype: int64

In [48]:
pd.value_counts(obj.values, sort=True)

a    3
c    3
b    2
d    1
dtype: int64

## Data Loading, Storage, and File Formats

pandas features a number of functions for reading tabular data as a DataFrame object, summarized in the table below. `read_csv` and `read_table` are likely the ones you’ll use the most.

| Function | Description
|----------| ----------
| read_csv | Load delimited data from a file, URL, or file-like object; use comma as default delimiter
| read_table | Load delimited data from a file, URL, or file-like object; use tab ('\t') as default delimiter
| read_fwf | Read data in fixed-width column format (i.e., no delimiters)
| read_clipboard | Version of read_table that reads data from the clipboard; useful for converting tables from web pages
| read_excel | Read tabular data from an Excel XLS or XLSX file
| read_hdf | Read HDF5 files written by pandas
| read_html | Read all tables found in the given HTML document
| read_json | Read data from a JSON (JavaScript Object Notation) string representation
| read_msgpack | Read pandas data encoded using the MessagePack binary format
| read_pickle | Read an arbitrary object stored in Python pickle format
| read_sas | Read a SAS dataset stored in one of the SAS system’s custom storage formats
| read_sql | Read the results of a SQL query (using SQLAlchemy) as a pandas DataFrame
| read_stata | Read a dataset from Stata file format
| read_feather | Read the Feather binary

To start practicing with this, make a file called ex1.csv in a folder "examples" with these lines:

In [52]:
! cat << EOF
a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo
EOF > l19_test.csv

<< was unexpected at this time.


NameError: name 'a' is not defined

In [53]:
df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


We could also have used read_table and specified the delimiter:

In [54]:
pd.read_table('examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


A file will not always have a header row. Consider this file, 'examples/ex2.csv':

To read this file, you have a couple of options. You can allow pandas to assign default column names, or you can specify names yourself:

In [55]:
pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [56]:
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Suppose you wanted the message column to be the index of the returned DataFrame. You can either indicate you want the column at index 4 or named 'message' using the index_col argument:

In [57]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


What about a file with variable whitespace separating the columns, like examples/ex3.txt

You can pass a regular expression as a delimiter for read_table. This can be expressed by the regular expression \s+, so we have then:

In [58]:
result = pd.read_table('examples/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


Because there was one fewer column name than the number of data rows, read_table infers that the first column should be the DataFrame’s index in this special case.

What if you have comment lines, like in examples/ex4.csv?

In [59]:
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [60]:
pd.read_csv('examples/ex4.csv', comment='#')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Let's handle some missing data, like in examples/ex5.csv

In [61]:
result = pd.read_csv('examples/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1.0,2,3.0,4,NaN
1,two,5.0,6,NaN,8,world
2,three,NaN,10,11.0,12,foo


### Writing Data to Text Format
Data can also be exported to a delimited format. Let's use what we just read in.

In [62]:
result.to_csv('examples/out.csv')

Missing values appear as empty strings in the output. You might want to denote them by some other sentinel value:

In [63]:
result.to_csv('examples/out2.csv', na_rep='NULL')

With no other options specified, both the row and column labels are written. Both of these can be disabled:

In [64]:
result.to_csv('examples/out3.csv', index=False, header=False)

You can also write only a subset of the columns, and in an order of your choosing:

In [65]:
result.to_csv('examples/out4.csv', index=False, columns=['a', 'b', 'c'])

**Up next**: Visualization!!